In [2]:
import helper
import mne
import scipy.io
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as spstats
import statsmodels.stats as stats

pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
from pathlib import Path, PurePath

%matplotlib
plt.rcParams.update({'font.size': 14})

Using matplotlib backend: Qt5Agg


In [3]:
# Parameters 

# Subject and task
subject = 'DiAs'
subject_id = '04'
proc = 'raw'
montage = 'preproc'
suffix = '_lnrmv.set'
task = 'stimuli'
run = '1'

# High frequency bands 
l_freq = 60
nband = 6 
band_size = 20 

# Epoch parameter

tmin = -0.5 # Prestimulus
tmax = 1.75 # Poststimulus

# Saving paramerters 

save2 = Path('~','projects','CIFAR','data_fun').expanduser()
task_save = 'stimuli'
suffix_place = '_epoch_place.mat'
suffix_face = '_epoch_face.mat'

In [4]:
raw, dfelec = helper.import_data(task=task, proc=proc, montage=montage, run=run, subject=subject, subject_id=subject_id, suffix=suffix)

/home/guime/projects/CIFAR/new_code/test/helper.py:26: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(fpath, preload=load)
/home/guime/projects/CIFAR/new_code/test/helper.py:26: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(fpath, preload=load)


In [4]:
HFB, raw_HFB = helper.HFB_raw(raw, l_freq=60, nband=6, band_size=20);

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 80 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1e+02 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)



/home/guime/projects/CIFAR/new_code/test/helper.py:86: RuntimeWarning: invalid value encountered in true_divide
  HFB = np.divide(HFB, np.mean(HFB, axis=1)[:,np.newaxis])


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.2e+02 - 1.4e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.4e+02 - 1.6e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Blackman window with 0.0017 passband ripple and 74 dB stopband attenuation
- Lower transition bandwidth: 10.00 Hz
- Upper transition bandwidth: 10.00 Hz
- Filter length: 251 samples (0.502 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.6e+02 - 1.8e+02 Hz

FIR 

In [5]:
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: ['-1', '-2', 'Face010', 'Face020', 'Face030', 'Face040', 'Face050', 'Face060', 'Face070', 'Place010', 'Place020', 'Place030', 'Place040', 'Place050', 'Place060', 'Place070', 'boundary']


In [6]:
place_id, face_id = helper.stim_id(event_id)

In [7]:
epochs = mne.Epochs(raw_HFB, events, event_id= event_id, tmin=tmin, tmax=tmax, baseline=None,preload=True)

59 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 59 events and 1126 original time points ...
1 bad epochs dropped


In [8]:
time = epochs.times

In [9]:
HFB_db = helper.HFB_norm(epochs, events, tmin)

/home/guime/projects/CIFAR/new_code/test/helper.py:153: RuntimeWarning: invalid value encountered in true_divide
  HFB_norm[i,j,:] = np.divide(HFB_epoch[i,j,:], baseline[i,j])# divide by baseline
/home/guime/projects/CIFAR/new_code/test/helper.py:158: RuntimeWarning: divide by zero encountered in log10
  HFB_db[i,j,:] = 10*np.log10(HFB_norm[i,j,:]) # transform into normal distribution


58 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


In [13]:
picks = 'LGRD1'

In [14]:
evok = HFB_db[face_id].copy().pick(picks).average()
ERP = evok.data

In [ ]:
plt.plot(time, ERP[0,:]) 

In [97]:
evok_po = HFB_db.copy().crop(tmin=0.1, tmax=0.5).average()
evok_pr = HFB_db.copy().crop(tmin=-0.5, tmax=-0.1).average()

/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [98]:
A_po = evok_po.data
A_pr = evok_pr.data

In [99]:
# compute cohen d
M1 = np.mean(A_po, axis=1)
M2 = np.mean(A_pr, axis=1)
std1 = np.std(A_po, axis=1)
std2 = np.std(A_pr, axis=1)
n1 = A_po.shape[1]
n2 = A_pr.shape[1]
std = np.sqrt(np.divide((n1-1)*std1**2+(n2-1)*std2**2,(n1+n2-2)))
cohen = np.divide(M1-M2, std)

/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/numpy/core/_methods.py:193: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in subtract
  if __name__ == '__main__':


In [129]:
from numpy import inf
M1[M1==-inf]=0
M2[M2 == -inf] =0

In [115]:
M1 = np.transpose(M1)

In [133]:
plt.hist(M1, bins = 80)

(array([ 1.,  0.,  1.,  1.,  3.,  5.,  9., 18., 18., 25., 25., 16.,  7.,
         3.,  1.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  1.,  1.,  2.,  1.,  2.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.]),
 array([-1.06253777, -0.94240607, -0.82227436, -0.70214266, -0.58201095,
        -0.46187925, -0.34174754, -0.22161584, -0.10148413,  0.01864757,
         0.13877928,  0.25891098,  0.37904269,  0.49917439,  0.6193061 ,
         0.7394378 ,  0.85956951,  0.97970121,  1.09983292,  1.21996462,
         1.34009633,  1.46022803,  1.58035974,  1.70049144,  1.82062315,
         1.94075485,  2.06088656,  2.18101826,  2.30114997,  2.42128167,
         2.54141337,  2.66154508,  2.78167678,  2.90180849,  3.02194019,
         3.1420719 ,  3.2622036

In [131]:
plt.hist(M1, bins=100)

(array([ 1.,  0.,  0.,  2.,  0.,  1.,  4.,  6.,  8., 16., 11., 18., 24.,
        20., 11.,  7.,  2.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  2.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,
         2.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([-1.06253777e+00, -9.66432406e-01, -8.70327042e-01, -7.74221678e-01,
        -6.78116315e-01, -5.82010951e-01, -4.85905587e-01, -3.89800223e-01,
        -2.93694859e-01, -1.97589495e-01, -1.01484131e-01, -5.37876747e-03,
         9.07265964e-02,  1.86831960e-01,  2.82937324e-01,  3.79042688e-01,
         4.75148052e-01,  5.71253416e-01,  6.67358780e-01,  7.63464143e-01,
         8.59569507e-01,  9.55674871e-01,  1.05178024e

In [119]:
sns.barplot(M1)

In [121]:
sns.distplot(M1, kde=False, rug=True);

In [90]:
Fsmall = len(cohen[cohen>0.2])/len(cohen)
Fmedium = len(cohen[cohen>0.5])/len(cohen)
Flarge = len(cohen[cohen>0.8])/len(cohen)

/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
for i in range(0,len(epochs.info['ch_names'])):
    print(stats.ttest_rel(A_po[i,:], A_pr[i,:]))

AttributeError: module 'statsmodels.stats' has no attribute 'ttest_rel'

In [24]:
tstat, pval, df = stats.weightstats.ttest_ind(A_po, A_pr)

AttributeError: module 'statsmodels.stats' has no attribute 'weightstats'

In [30]:
from statsmodels.stats.weightstats import ttest_ind
tstat, pval, df = ttest_ind(A_po, A_pr)

/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/statsmodels/stats/weightstats.py:138: RuntimeWarning: invalid value encountered in subtract
  return self.data - self.mean
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/statsmodels/stats/weightstats.py:617: RuntimeWarning: invalid value encountered in subtract
  tstat = (value1 - value2 - diff) / std_diff
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/guime/anaconda3/envs/neuro/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [51]:
pval = [0]*len(epochs.info['ch_names'])
degf = [0]*len(epochs.info['ch_names'])
tstat = [0]*len(epochs.info['ch_names'])
for i in range(0,len(epochs.info['ch_names'])):
    tstat[i], pval[i], degf[i] = ttest_ind(A_po[i,:], A_pr[i,:])

In [77]:
from statsmodels.stats.multitest import fdrcorrection, multipletests

In [71]:
pval = np.nan_to_num(pval)
reject, pval_correct = fdrcorrection(pval, alpha=0.01)

In [86]:
from statsmodels.stats.multitest import fdrcorrection, multipletests
reject = multipletests(pval, alpha=0.01, method='fdr_bh')

In [5]:
dfelec

,Unnamed: 0,Brodman,ROI_DK,electrode_name,hemisphere,isdepth,X,Y,Z
0,0,BA2,ctx-lh-superiortemporal,LGRD8,lh,0,-58.870430,-37.561195,4.570933
1,1,BA2,ctx-lh-supramarginal,LGRD16,lh,0,-58.901756,-34.131065,10.976174
2,2,BA2,ctx-lh-supramarginal,LGRD24,lh,0,-56.349739,-30.529045,17.004347
3,3,BA2,ctx-lh-supramarginal,LGRD32,lh,0,-50.230492,-27.593634,25.835140
4,4,BA2,ctx-lh-postcentral,LGRD40,lh,0,-48.351048,-23.122154,31.483007
5,5,BA3b,ctx-lh-postcentral,LGRD48,lh,0,-41.916389,-18.583757,39.394634
6,6,BA2,ctx-lh-middletemporal,LGRD56,lh,0,-61.242317,-23.574440,-8.700449
7,7,BA3b,ctx-lh-middletemporal,LGRD64,lh,0,-59.718948,-22.171385,-16.964657
8,8,BA2,ctx-lh-superiortemporal,LGRD7,lh,0,-63.609299,-31.176781,-0.664145
9,9,BA2,ctx-lh-superiortemporal,LGRD15,lh,0,-60.632751,-25.841021,7.088796
